In [1]:
import os
import pandas as pd
import gzip
import json
from tqdm.notebook import tqdm

# see contabo server
data_path = "cache_payload/stars"

In [ ]:
data = []
for date in tqdm(os.listdir(data_path)):
# for date in os.listdir(data_path):
    # print(date)
    for pkg_date_snapshot in os.listdir("/".join([data_path, date])):
    # for pkg_date_snapshot in os.listdir("/".join([data_path, date]))[:5]:
        slug = pkg_date_snapshot.replace(f"{date}_", "").replace(".json.gz", "")
        # try:
            # user, repo = slug.split("_", 1)
            # user, repo = slug.split("_")
        # except ValueError:
        #     print(f"Error with slug: {slug}")
    # for pkg_date_snapshot in os.listdir("/".join([data_path, date])):
        full_path = "/".join([data_path, date, pkg_date_snapshot])
        if full_path.endswith(".json.gz"):
            with gzip.open(full_path, "rb") as f:
                stargazer_data = json.load(f)
            stars = len(stargazer_data)
            # print(stars)
            
            # data.append([slug, date, stars])
            data.append([slug, date, stars])            

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
df = (
    pd.DataFrame(data, columns=["slug", "date", "stars"])
    # pd.DataFrame(data, columns=["slug", "date", "user", "repo", "stars"])
    .sort_values(["slug", "date"], ignore_index=True)
)
# df.to_csv("repo_stars_timeseries.csv", index=False)
df

In [ ]:
# # Shows what an empty payload contains:
# _path = "cache_payload/stars/2023-05-28/2023-05-28_7RedViolin_pySigma-backend-sentinelone.json.gz"
# with gzip.open(_path, "rb") as f:
#     json_data = json.load(f)
# json_data

In [ ]:
# # Shows what a (non-empty) payload contains:
# _path = "cache_payload/stars/2023-05-28/2023-05-28_0riion_py-sls-lambda-toolkit.json.gz"
# with gzip.open(_path, "rb") as f:
#     json_data = json.load(f)
# json_data

In [ ]:
# Read in 100 random sample
df_random_sample = (
    pd.read_csv("../input/random_sample.csv")
    .assign(slug=lambda df: df["github"].str.replace("https://github.com/", ""))
    .assign(slug=lambda df: df["slug"].apply(lambda x: x[:-1] if x.endswith("/") else x))
    .assign(slug=lambda df: df["slug"].str.lower().str.strip())
    .assign(fileslug=lambda df: df["slug"].str.replace("/", "_"))
    .assign(treated=1)
    # Assign the two types
    .assign(boughtstars=lambda df: (df.index < 25).astype(int))
    .assign(treated2=lambda df: df["boughtstars"] + df["treated"])
)
df_random_sample.head(3)

In [ ]:
df.to_csv("../output/repo_stars_timeseries.csv", index=False)